#Creating Dta set


In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

num_samples = 1000

# More realistic features
square_footage = np.random.normal(2000, 500, num_samples)
num_bedrooms = np.random.randint(1, 6, num_samples)
distance_to_city = np.random.normal(10, 4, num_samples)

# Realistic target: house price
# Price = 100*sqft + 20000*bedrooms - 5000*distance + noise
price = (100 * square_footage + 
         20000 * num_bedrooms - 
         5000 * distance_to_city + 
         np.random.normal(0, 25000, num_samples))

df = pd.DataFrame({
    'square_footage': square_footage,
    'num_bedrooms': num_bedrooms,
    'distance_to_city': distance_to_city,
    'house_price': price
})

df.to_csv('data.csv', index=False)
print("Housing dataset created!")
print(df.head())

Housing dataset created!
   square_footage  num_bedrooms  distance_to_city    house_price
0     2248.357077             4          9.491474  274017.935657
1     1930.867849             1          6.491259  198063.314927
2     2323.844269             3          7.890856  236893.676654
3     2761.514928             5          7.950681  347773.437843
4     1882.923313             3          8.969839  198708.257146


#Import Library and setup

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
df = pd.read_csv('data.csv')
X = df.iloc[:, :3].values  # First 3 columns as input features
y = df.iloc[:, 3].values.reshape(-1, 1)  # 4th column as target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

#Convert datas to tensors

In [5]:
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

#Define Model

In [6]:
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear = nn.Linear(input_dim,1)

    def forward(self,x):
        return self.linear(x)

model = LinearRegressionModel(3).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)



#Training Loop

In [7]:
epochs =1000
for epoch in range (epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs ,y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1)%100 == 0 :
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.1754
Epoch [200/1000], Loss: 0.1359
Epoch [300/1000], Loss: 0.1357
Epoch [400/1000], Loss: 0.1357
Epoch [500/1000], Loss: 0.1357
Epoch [600/1000], Loss: 0.1357
Epoch [700/1000], Loss: 0.1357
Epoch [800/1000], Loss: 0.1357
Epoch [900/1000], Loss: 0.1357
Epoch [1000/1000], Loss: 0.1357


#Evaluation

In [8]:
model.eval()
with torch.no_grad(): ## This disables gradient computation
    predictions = model(X_test)
    test_loss = criterion(predictions ,y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.1434
